## Projeto nº 4: Aprendizagem Automática

### Introdução à Inteligência Artificial 2020/2021

### Etapa 1:
Para carregar os dados é utilizada a função load_data fornecida no módulo utilsAA

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris # conjunto de dados
from sklearn.tree import DecisionTreeClassifier, plot_tree # árvore de decisão
from sklearn.neighbors import KNeighborsClassifier # k-NN
from sklearn.model_selection import train_test_split, cross_val_score # cross-validation
import numpy as np 
import matplotlib.pyplot as plt # gráficos
from utilsAA import * # módulo distribuido com o guião com funções auxiliares
from sklearn import preprocessing

airline_data, airline_target, airline_dclass, airline_tclass = load_data('airline.csv')


## Etapa 2:

Começamos por fazer uma decision tree primeiro

Para passar de String a inteiros o que fazemos é o seguinte: vamos buscar a coluna i, que são todos elementos de uma classe de dados, vemos quantos valores diferentes existem (utilizando o unique) e guardamos os vários valores diferentes num array. De seguida vamos ver em que posição se encontra cada um desses elementos da nossa data no nosso array obtido pelo unique. A posição em que ele se encontra passa a ser o valor que vai ficar. Por exemplo: em relação a male e female, o array obtido pelo unique tem male na posição 1 e female na posição 0, portanto todos os males são substituídos 1 e female por 0.

Como a nosso ver o ID do cliente não influencia a sua classificação decidimos remover a coluna com os ID's dos clientes (np.delete(array, subarray, 0 ou 1 caso seja linha ou coluna)


In [2]:
for i in range(len(airline_data[0])):
    if type(airline_data[0][i]) == str:
        airline_data[:,i] = encode_feature(airline_data[:,i])
airline_tdata = airline_data
airline_tdata = np.delete(airline_data, 0, 1)

## Etapa 3

### Melhor modelo
Para descobrirmos o melhor modelo testámos os 2 modelos existentes, K-Vizinhos e árvore de decisão.

#### Árvore de Decisão
Em relação à árvore de decisão fazemos 3 for's até 20 para os vários parâmetros que vamos alterar: max_depth, min_samples_split e min_samples_leaf. Para cada valor no for, criamos uma árvore de decisão para i, j e k para os parâmetros e com o cross_val_score vamos calcular o score dessa árvore de decisão. Com os melhores valores vamos então então criar a árvore de decisão final e depois usamos o cross_val_score para obter o seu score. Como cv = 10, o cross_val_score devolve 10 scores, e o np.std(scores) dá uma média do score e o np.mean(scores) dá o erro associado

In [25]:
scores = None
best = None
best_i = 0
best_j = 0
best_k = 0
for i in range(1,10):
    for j in range(2,10):
        for k in range(1,10):
            dtc = DecisionTreeClassifier(criterion='entropy', max_depth = i, min_samples_split = j, 
                                         min_samples_leaf = k)
            
            scores = cross_val_score(dtc, X=airline_tdata, y=airline_target, cv=10, n_jobs=-1)
            t = np.mean(scores)
            if best == None or t > best:
                best = t
                best_i = i
                best_j = j
                best_k = k

print(best_i)
print(best_j)
print(best_k)
dtc = DecisionTreeClassifier(criterion='entropy', max_depth = best_i, min_samples_split = best_j, 
                                         min_samples_leaf = best_k)
dtc.fit(airline_tdata, airline_target)
scores = cross_val_score(dtc, X=airline_tdata, y=airline_target, cv=10, n_jobs=-1)
print(np.std(scores))
print(np.mean(scores))


9
2
1
0.03551128494512635
0.8959207459207459


#### K-Vizinhos
Em relação aos K-Vizinhos fazemos 1 for até 100 para o único parâmetro que vamos alterar: n_neighbors. Para cada valor no for, criamos um KVizinhos para o parâmetro e com o cross_val_score vamos calcular o score desse K-Vizinhos. Vamos então buscar o melhor i (o que deu um melhor resultado) e criamos um K-Vizinhos final com esse valor. Como cv = 10, o cross_val_score devolve 10 scores, e o np.std(scores) dá uma média do score e o np.mean(scores) dá o erro associado

In [26]:
scores = None
best = None
best_i = 0
best_erro = None
for i in range(1, 100): 
    clf = KNeighborsClassifier(n_neighbors=i)
    
    scores = cross_val_score(clf, X=airline_data, y=airline_target, cv=10, n_jobs=-1)
    t = np.mean(scores)
    if best == None or t > best:
        best = t
        best_i = i
        best_erro = np.std(scores)

clf = KNeighborsClassifier(n_neighbors=best_i)

scores = cross_val_score(clf, X=airline_data, y=airline_target, cv=10, n_jobs=-1)
print(best_i)
print(np.mean(scores))
print(np.std(scores))

94
0.5758241758241758
0.015327821210921386


### Análise dos resultados
Como é possível reparar, o melhor método para fazer previsões é a árvore de decisão 0.90% +-0.03 enquanto que o K-Vizinhos tem apenas 0.58% +- 0.01

Então para fazer a previsão do teste, vamos utilizar a árvore de decisão final que foi criada utilizando os melhores valores.

### Previsões
Utilizando o método save_data e tratando a data do teste da mesma forma que foi tratada a data do airline, vamos então fazer a previsão utilizando a árvore de decisão criada a cima, não esquecendo de treinar a árvore de decisão com os dados do airline.csv

O resultado é guardado no ficheiro IIA2020-proj4-29.csv

In [27]:
test_data, test_dclass = load_data('test.csv', True)
test_tdata = np.delete(test_data, 0, 1)

for i in range(len(test_tdata[0])):
    if type(test_tdata[0][i]) == str:
        test_tdata[:,i] = encode_feature(test_tdata[:,i])


predict = dtc.predict(test_tdata)

save_data("IIA2021-proj4-29.csv", predict)